In [2]:
import pandas as pd
import numpy as np
import os
from datetime import *
from fn import *
from o_rpa_def import *
from o_rpa_defonly import *
import o_time as oT
from collections import defaultdict

def colchk(df):
    mcols = ['EQUIPMENTKEY','SITECODE','SUMMARY','ALERTKEY','LASTOCCURRENCE','CLEARTIMESTAMP']
    ocols = ['RESOURCE','CUSTOMATTR15','SUMMARY','ALERTKEY','LASTOCCURRENCE','CLEARTIMESTAMP']
    df = df.rename (columns=str.upper)
    cols = df.columns.to_list()
    if cols.count('SITECODE') != 0:
        df = df.rename(columns={'SITECODE':'CUSTOMATTR15'})
    if cols.count('EQUIPMENTKEY') != 0:
        df = df.rename(columns={'EQUIPMENTKEY':'RESOURCE'})
    for i in ocols:
        if ocols.count(i) == 0:
            print('must have column needs in table: but missing !',chr(10),mcols,chr(10),'exiting .....')
            exit(0)
    else:
        sx = chrstream()
        omnm(sx)
        print(chr(10))
        return df

def nested_dic_add(dc, k, v):
    #map based on child dictionary and add new key value/append by key matching on clild
    if len(dc)>0:
        ln = len(list(dc))
        for i in dc:
            if k in list(dc[i]):
                dc[i][k] = v if not list(dc[i]) else dc[i].get(k, []) + [v]
                return dc
        else:
            if isinstance(v, list):
                dc[ln+1] = {k:v}
            else:
                v1 = [v]
                dc[ln+1] = {k:v1}
            return dc
    else:
        if isinstance(v, list):
            dc[1] = {k:v}
        else:
            v1 = [v]
            dc[1] = {k:v1}
        return dc    

def mprep(df, dic, oncol):
    heap = ""
    gdc = defaultdict(dict)
    for k, v in dic.items():
        dff = df[df[k].isin(v)]
        dff = dff.reset_index()
        for i in range(len(v)):
            hp = str(k) + "|" + str(dff.shape[0]) + chr(10) + dff[oncol].str.contains(v[i]).cat(sep=chr(10))
            if heap == "":
                heap = hp
            else:
                heap = heap + chr(10) + chr(10) + hp
        else:
            print(heap)
    
class o_rpa:
    def __init__(self, mdf):
        self.odb = omdb()
        self.df0 = colchk(mdf)
        self.df1 = catmap(self.df0, self.odb)
        self.df = self.df1        
        self.pickcols=[]
        self.msgthread = defaultdict(dict)
        self.mth = []
        self.cond = defaultdict(dict)
    def regionwise(self, Name=[]):
        self.msgthread = list(zn_dic())
    def techwise(self):
        self.msgthread = {'CAT': {'2G':"","3G":"","4G":""}}
    def pickcol(self, col, fmt = "CODE: Last Ocurrance"):
        if col in self.df.columns.to_list():
            aa = self.pickcols + [col]
            self.pickcols = aa
    def csvmap(self, csvpath, match_col_name, column_to_pick = []):
        try:
            try:
                dff = pd.read_csv(csvpath)
            except:
                dff = csvpath
            if column_to_pick is not None:
                dff = dff[column_to_pick]
            ndf = self.df
            self.df = ndf.merge (dff, on=match_col_name)
        except:
            print('csvpath not found')
    def getdf(self, current=True):
        if current==False:
            return self.df0
        else:
            return self.df
    def sample(self):
        print(self.df.head(5))
    def summary(self):
        print("COLUMN:", self.df.columns, chr(10))
        print("Current Row: ", self.df.shape[0],'-',"Row Main: ", self.df1.shape[0], chr(10))
        print("filtering conditions : ", dict((k, v) for k, v in self.cond.items()))
        print("msgformat by pickcols: ", 'CUSTOMATTR15','LASTOCCURRENCE')
        print("msgthread: ", self.msgthread)
    def condition(self,colname,colval):
        ndc = nested_dic_add(self.cond,colname,colval)
        self.cond = ndc.copy()
    def rwise(self, whichzn=False):
        if whichzn == False:
            whichzn = "ALL"
        if len(self.cond) == 1:
            colsMain = list(self.cond[1])
            rval = parsing(self.df, whichzn,[],colsMain[0], self.cond[1].get(colsMain[0]), False, False)
        elif len(self.cond) == 2:
            colsMain = list(self.cond[1])
            cl = list(self.cond[2])
            rval = parsing(self.df, whichzn,self.pickcols,colsMain[0], self.cond[1].get(colsMain[0]), cl[0], 
                           self.cond[2].get(cl[0]))
        else:
            print('under development')
    def regionwise_count(self, list_cat):
        rv = zonewise_count(self.df1, list_cat)
        print('----------------------------------------')
        return rv
    def update_cond_1(self, ky, vl):
        self.cond.setdefault(ky, []).append(vl)
    def update_cond_2(self, ky, vl):
        if isinstance(vl,list):
            self.cond[ky] = vl if not list(self.cond) else self.cond.get(ky, []) + vl #check and add [best]
        else:
            vlx = [vl]
            self.cond[ky] = vlx if not list(self.cond) else self.cond.get(ky, []) + vlx #check and add [best]
    def timecal(self, start_time_colname, end_time_colname=False):
        self.df = oT.dfdiff(self.df, start_time_colname, end_time_colname)
        print("new column name is 'DUR'")
    def timefmt(self, colname, fmt="%Y/%m/%d %H:%M:%S"):
        if len(fmt) <= 9:
            self.df[colname] = self.df.apply(lambda x: oT.sec_to_dur(x[colname]), axis = 1)
            print(self.df[colname])
        else:
            self.df[colname] = oT.datetime_convert_format(self.df, colname, fmt)
            print(self.df[colname])
    def genx(self):
        ln = len(self.lsky)
        ls = []
        cnt = 0
        if ln>=2:
            for i in range(ln):
                cnt = cnt + 1
                if len(ls) == 0:
                    ls = joinls(self.lsval[i],self.lsval[cnt])
                elif len(ls) != 0 and cnt<ln:
                    lss = []
                    lss = joinls(ls,self.lsval[cnt])
                    ls = lss
            else:
                print(ls)


df = pd.read_csv(os.getcwd() + "\\sclick.csv")  # data source, 
dfp1p2 = pd.read_csv (os.getcwd() + "\\site_p1p2.csv")
xx = o_rpa(df)
xx.timecal('LASTOCCURRENCE')
xx.timefmt('DUR','%H:%M:%S')
xx.update_cond_2('CAT',['MF','DL'])
xx.update_cond_2('sZone',['COM'])
rval = xx.rwise()
xx.genx()
#ST = xx.regionwise_count(['2G','3G','4G','MF','DL'])
#print(ST)
#dc = {'sZone':{0:{'COM':{'CAT':{0:'2G'}}}}}
#print(dc)
#slicedf(dx,dc)




- - - - - - - - - - - - - - - - - -
[      ____  __  __   _          
      / __ \|  \/  | | |         
     | |  | | \  / | | |         
     | |  | | |\/| | | |         
     | |__| | |  | | | |         
      \____/\_|  |_| |_|        ]

- - - - - - - - - - - - - - - - - -


new column name is 'DUR'
0       0:22:27
1        0:6:37
2        0:1:37
3        0:7:32
4        0:2:40
         ...   
2438     0:16:4
2439    0:16:12
2440     0:16:5
2441     0:16:3
2442     0:16:3
Name: DUR, Length: 2443, dtype: object


IndexError: list index out of range

In [20]:
def twocond(dc1):
    dc = {}
    for k,v in dc1.items():
        if type(dc1[k].values) is dict:
            towcond(dc1[k].values)
        else:
            xx = dc1[k]
            dc[xx.key] = list(xx.values())
    else:
        print(dc)


nls = [['2G', '3G', '4G'],['AT','RB']]
ky = ['one', 'two', 'three']
D = {k: v for k, v in zip(ky, [x for x in nls])}
L = [number for list in nls for number in list]
nesdic = {1: {"A1": "100"}, 2: {"A2": "200"}, 3: {'A3':"300"}}
print(L)


['2G', '3G', '4G', 'AT', 'RB']
